In [66]:
from scipy.stats import chi2
from numpy import random, bincount
from IPython.display import display, Math

---
# Ejercicio 6
Un escribano debe validar un juego en cierto programa de televisión. El mismo consiste
en hacer girar una rueda y obtener un premio según el sector de la rueda que coincida con una aguja.
Hay 10 premios posibles, y las áreas de la rueda para los distintos premios, numerados del 1 al 10, son respectivamente:
$$
31\%, \quad 22\%,\quad 12\%, \quad 10\%,\quad  8\%,\quad 6\%,\quad 4\%,\quad 4\%,\quad 2\%,\quad 1\%
$$

Los premios con número alto (e.j. un auto 0Km) son mejores que los premios con número bajo (e.j. 2x1
para entradas en el cine). El escribano hace girar la rueda hasta que se cansa, y anota cuántas veces sale cada sector. Los resultados, para los premios del 1 al 10, respectivamente, son:
$$
188, \quad 138, \quad 87,\quad 65,\quad 48,\quad 32,\quad 30,\quad 34,\quad 13,\quad 2
$$

Esto implica que se ordenan a *mayor prioridad números del fondo*:
- 31\% -> 188 veces **(2x1 para entradas en el cine)**
- 22\% -> 138 veces
- 12\% -> 87  veces 
- 10\% -> 65  veces 
-  8\% -> 48  veces 
- 6\%  -> 32  veces 
- 4\%  -> 30  veces 
- 4\%  -> 34  veces 
- 2\%  -> 13  veces 
- 1\%  -> 2   veces **(un auto 0Km)**

Dado que los datos son una muestra de observaciones independientes discretas, vamos a utilizar el `Test de bondad de ajuste de Pearson`

a) Construya una tabla con los datos disponibles

In [40]:
PROBABILITY_VALUES = [
    0.31, 0.22, 0.12,
    0.10, 0.08, 0.06,
    0.04, 0.04, 0.02,
    0.01
]

OBSERVED_FRECUENCIES = [
    188, 138, 87,
    65,  48,  32,
    30,  34,  13,
    2
]

NSAMPLES = sum(OBSERVED_FRECUENCIES)

T_statistic = 0

print(f"{'i':^2} | {'Nᵢ':^4} | {'pᵢ':^6} | {'Nsamples × pᵢ':^7} | {'((Nᵢ - Nsamples × pᵢ)^2) / Nsamples × pᵢ':^25} |")
print("-" * 78)

for i, (obs, prob) in enumerate(zip(OBSERVED_FRECUENCIES, PROBABILITY_VALUES)):
    expected = round(NSAMPLES * prob, 4)
    term = round((obs - expected) ** 2 / expected, 4)
    T_statistic += term
    print(f"{i:^2} | {obs:^4} | {prob:^6.2f} | {expected:^13} | {T_statistic:^40.4f} |")

print("-" *78)
print(f"Estadístico de Pearson: {T_statistic}".center(78) + "|" )
print("-" *78)


i  |  Nᵢ  |   pᵢ   | Nsamples × pᵢ | ((Nᵢ - Nsamples × pᵢ)^2) / Nsamples × pᵢ |
------------------------------------------------------------------------------
0  | 188  |  0.31  |    197.47     |                  0.4541                  |
1  | 138  |  0.22  |    140.14     |                  0.4868                  |
2  |  87  |  0.12  |     76.44     |                  1.9456                  |
3  |  65  |  0.10  |     63.7      |                  1.9721                  |
4  |  48  |  0.08  |     50.96     |                  2.1440                  |
5  |  32  |  0.06  |     38.22     |                  3.1563                  |
6  |  30  |  0.04  |     25.48     |                  3.9581                  |
7  |  34  |  0.04  |     25.48     |                  6.8070                  |
8  |  13  |  0.02  |     12.74     |                  6.8123                  |
9  |  2   |  0.01  |     6.37      |                  9.8102                  |
-----------------------------------------

---
b) Diseñe una prueba de hipótesis para determinar si la rueda es justa.

Como *el escribano* durante el testeo obtuvo esas frecuencias con dicho porcentaje de probabilidad, asumimos que una rueda **es justa** si vienen dados por los resultados obtenidos. Es decir, un auto 0km tiene probabilidad de salir del 1%.

Dicho esto tenemos:
> "$H_0$: `La rueda es justa`, es decir, los datos observados provienen de una distribución con las probabilidades dadas:
> $$ H_0 : P(i)=p_i \quad \text{para cada } i=0,…,9$$ 


---
c) Defina el p-valor a partir de la hipótesis nula

Como tenemos que el estadístico de Pearson viene dado por:
$$
\boxed{T = 9.8102}
$$
Como hemos considerado 10 términos en el estadístico T, entonces debemos testear con una distribución $\chi_{10-1}^2$. En particular:
$$
p_{valor} = P(\chi_{9}^2 \geq 9.8102) 
$$


---
d) Calcule el p-valor bajo la hipótesis de que la rueda es justa, usando la aproximación chi cuadrado

In [44]:
P_VALUE = 1 - chi2.cdf(round(T_statistic, 4), df=9)
round(float(P_VALUE), 4)

0.3661

$$
p_{valor} = P(\chi_{9}^2 \geq 9.8102) \approx 0.3661
$$

Para un nivel de confianza del 95%, no hay evidencia suficiente para rechazar Ho porque el $p_{valor}$ es menor al 5%.

---
e) Calcule el p-valor bajo la hipótesis de que la rueda es justa, usando una simulación.

In [61]:
def Pearson_statistic(freqs, probability_values, Nsamples):
    T_statistic = 0
    for obs, prob in zip(freqs, probability_values):
        expected = Nsamples * prob
        term = (obs - expected) ** 2 / expected
        T_statistic += term
    return T_statistic

In [70]:
p_value = 0
t0 = T_statistic
Nsim = 10_000
possible_frequencies = list(range(0, len(OBSERVED_FRECUENCIES)))

for _ in range(Nsim):
    simulated_data = random.choice(possible_frequencies, size=NSAMPLES, p=PROBABILITY_VALUES)
    simulated_freqs = bincount(simulated_data, minlength=len(OBSERVED_FRECUENCIES))
    T = Pearson_statistic(simulated_freqs, PROBABILITY_VALUES, NSAMPLES)

    if T >= t0:
        p_value += 1

p_value /= Nsim
display(Math(fr"p_{{\text{{valor}}}} = P(\chi^2_9 \geq 9.8102) \approx {p_value:.4f}"))

<IPython.core.display.Math object>